<a href="https://colab.research.google.com/github/keshvi-srivastava/star-wars-dialogue-generation/blob/main/Model3_Sliding_window_bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model to generate a sequence of following words:
1. Convert the data into token list
2. Convert data to token sentences with sliding windows
3. Encode the sentence
4. Simple LSTM model
5. Create a bidirectional model

- Makes sentences sequence from the whole token list
- Make a sliding window of size 5 each

Reference:

https://medium.com/@plusepsilon/the-bidirectional-language-model-1f3961d1fb27

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd
import re
from numpy import array
from pickle import dump
import string
from random import randint
from pickle import load
from tensorflow.keras.layers.experimental import preprocessing

from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import categorical_accuracy

#import spacy, and spacy french model
# spacy is used to work on text

#import other libraries
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle

In [ ]:
path_to_file = '/content/drive/MyDrive/SNLP Project/Filtered_Data/'

data = pd.DataFrame(columns = ['character', 'dialogue'])

for file in os.listdir(path_to_file):
    print(file)
    df = pd.read_csv(path_to_file+file)
    data = data.append(df, ignore_index=True)

data['character'] = data["character"].str.lower()

data['character'] = data.character.replace("anakin", "vader", regex=True)
data['character'] = data.character.replace("obi-wan", "ben", regex=True)
data['character'] = data.character.replace("c-3po", "threepio", regex=True)

unique_characters = data.character.unique()

data_dict = data.groupby('character')['dialogue'].apply(lambda g: g.values.tolist()).to_dict()

SW_EpisodeI.csv
SW_EpisodeII.csv
SW_EpisodeIII.csv
SW_EpisodeIV.csv
SW_EpisodeV.csv
SW_EpisodeVI.csv


In [ ]:
def preprocess_text(sen):

    # Remove numbers
    sentence = re.sub(" \d+", " ", sen)

    # # Single character removal
    # sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sen)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    # Remove ....
    sentence = re.sub('\.+', ' ', sentence)

    # Remove punctuations
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)

    # Lower case
    sentence = sentence.lower()

    # Return a list of tokens (words)
    sentence = sentence.split()

    return sentence

In [ ]:
obi_wan_tokens = [preprocess_text(row) for row in data_dict['ben']]
print(obi_wan_tokens[:5])

obi_wan_data = [' '.join(row) for row in obi_wan_tokens]
print(obi_wan_data)

obi_wan_token_list = [item for sublist in obi_wan_tokens for item in sublist]
print(obi_wan_token_list)

print("Total # of tokens(words)")
print(len(obi_wan_token_list))

print("Total # of unique tokens(words)")
print(len(set(obi_wan_token_list)))

[['i', 'have', 'a', 'bad', 'feeling', 'about', 'this'], ['its', 'not', 'about', 'the', 'mission', 'master', 'its'], ['master', 'yoda', 'says', 'i', 'should', 'be', 'mindful', 'of', 'the', 'future'], ['yes', 'master', 'how', 'do', 'you', 'think', 'the', 'trade', 'viceroy', 'will', 'deal', 'with'], ['offhand', 'id', 'say', 'this', 'mission', 'is', 'past', 'the', 'negotiaion', 'stage']]
['i have a bad feeling about this', 'its not about the mission master its', 'master yoda says i should be mindful of the future', 'yes master how do you think the trade viceroy will deal with', 'offhand id say this mission is past the negotiaion stage', 'they have shield generators', 'its an invisible army', 'you were right about one thing master the negotiations were', 'sorry master the water fried my weapon', 'yes master', 'whats this', 'when they find us they will crush us grind us into little', 'dont worry this has not been our day for warm welcomes', 'after those droids take control of the surface the

In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(obi_wan_token_list)
unique_words = set(obi_wan_token_list)
sequences_tokenised = tokenizer.texts_to_sequences(obi_wan_tokens)

vocab_size = len(unique_words)+1
n_sentences = len(obi_wan_tokens)

In [ ]:
print(unique_words)
print(sequences_tokenised)

print(vocab_size)
print(n_sentences)

{'eighteen', 'half', 'myself', 'short', 'trying', 'democracy', 'command', 'continuingyou', 'split', 'clumsy', 'speeches', 'son', 'live', 'speak', 'absolutely', 'warning', 'kenobi', 'atmosphere', 'weakminded', 'stuck', 'thousand', 'hidden', 'wounds', 'created', 'uncle', 'she', 'tradefederation', 'recalibrated', 'around', 'stake', 'isn\x92t', 'hoping', 'minister', 'storms', 'age', 'sixtyforty', 'own', 'fancy', 'luck', 'fund', 'carefullytell', 'wastes', 'feelingsget', 'sidious', 'fact', 'theyve', 'over', 'starships', 'theold', 'flying', 'different', 'sieges', 'perhaps', 'investigate', 'downstairs', 'regroup', 'immunity', 'deliveries', 'thoughts', 'topof', 'representative', 'value', 'himself', 'time', 'computer', 'frowningwere', 'jedis', 'cling', 'briefing', 'iam', 'involved', 'you', 'luke', 'seduced', 'stressed', 'burden', 'ideals', 'avoiding', 'bigger', 'unconscious', 'yours', 'lightly', 'princess', 'cato', 'quigon', 'warilyim', 'like', 'focused', 'he', 'vos', 'quickly', 'points', 'survi

In [ ]:
#Create sliding windows
seq_length = 5
sequences_step = 1
dataX = []
dataY = []
for dialogue in sequences_tokenised:
  window = []
  for i in range(len(dialogue)-5):
    dataX.append(dialogue[i:i+5])
    dataY.append(dialogue[i+5])


In [ ]:
print(len(dataX))
print(len(dataY))
print(dataY[1450])

3596
3596
72


In [ ]:
X = np.zeros((len(dataX), seq_length, vocab_size+1), dtype=np.bool)
y = np.zeros((len(dataX), vocab_size+1), dtype=np.bool)
for i, sentence in enumerate(dataX):
  for t, word in enumerate(sentence):
    X[i, t, word] = 1
  y[i, dataY[i]] = 1

In [ ]:
print(X.shape)
print(y.shape)

(3596, 5, 1452)
(3596, 1452)


In [ ]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    print("model built!")
    return model

In [ ]:
rnn_size = 256 # size of RNN
seq_length = 5 # sequence length
learning_rate = 0.001 #learning rate

md = bidirectional_lstm_model(seq_length, vocab_size+1)
md.summary()

Build LSTM model.
model built!
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 512)               3500032   
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1452)              744876    
_________________________________________________________________
activation_4 (Activation)    (None, 1452)              0         
Total params: 4,244,908
Trainable params: 4,244,908
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 32 # minibatch size
num_epochs = 50 # number of epochs

callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath="./" + 'my_model_gen_sentences.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           monitor='val_loss', verbose=0, mode='auto', period=2)]
#fit the model
history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
                 callbacks=callbacks,
                 validation_split=0.1)

#save the model
md.save("./" + 'my_model_generate_sentences.h5')

Epoch 1/50
102/102 [==============================] - 13s 105ms/step - loss: 6.9393 - categorical_accuracy: 0.0440 - val_loss: 6.2859 - val_categorical_accuracy: 0.0556
Epoch 2/50
102/102 [==============================] - 10s 101ms/step - loss: 5.9615 - categorical_accuracy: 0.0598 - val_loss: 6.3316 - val_categorical_accuracy: 0.0556
Epoch 3/50
102/102 [==============================] - 10s 101ms/step - loss: 5.8089 - categorical_accuracy: 0.0519 - val_loss: 6.5960 - val_categorical_accuracy: 0.0556
Epoch 4/50
102/102 [==============================] - 10s 102ms/step - loss: 5.7636 - categorical_accuracy: 0.0555 - val_loss: 6.6038 - val_categorical_accuracy: 0.0583
Epoch 5/50
102/102 [==============================] - 10s 101ms/step - loss: 5.6120 - categorical_accuracy: 0.0474 - val_loss: 6.6740 - val_categorical_accuracy: 0.0556


In [ ]:
model = load_model("./" + 'my_model_generate_sentences.h5')

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
reverse_word_dict = {v: k for k, v in tokenizer.word_index.items()}

In [ ]:
words_number = 5 # number of words to generate
seed_sentences = "i have a feeling" #seed sentence to start the generating.

#initiate sentences
generated = ''

#we shate the seed accordingly to the neural netwrok needs:
# for i in range (seq_length):
#     sentence.append("a")

seed = seed_sentences.split()
sentence = tokenizer.texts_to_sequences(seed)

print(sentence)

[[4], [12], [5], [276]]


In [ ]:

# for i in range(len(seed)):
#     sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += seed_sentences

#the, we generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size+1))
    for t, word in enumerate(sentence):
        x[0, t, word] = 1.

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.33)
    next_word = reverse_word_dict[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

#print the whole text
print(generated)

0 [4]
1 [12]
2 [5]
3 [276]
0 [12]
1 [5]
2 [276]
3 the


IndexError: ignored

In [ ]:
words_number = 5 # number of words to generate
seed_sentences = "he is your father luke" #seed sentence to start the generating.

#initiate sentences
generated = ''
sentence = []

#we shate the seed accordingly to the neural netwrok needs:
for i in range (seq_length):
  sentence.append("a")

print(sentence)
seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)

#the, we generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size+1))
    for t, word in enumerate(sentence):
        x[0, t, tokenizer.word_index[word]] = 1.

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.33)
    next_word = reverse_word_dict[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

#print the whole text
print(generated)

['a', 'a', 'a', 'a', 'a']
he is your father luke you all with of are
